# Qiime 2 Analysis

### Analysis was conducted with Qiime 2 version 2017.10

This notebook contains Qiime commands (run on the command line) for data analysis presented in the paper. Input files for analysis are provided in the "Raw_Files" folders corresponding to each study. They contain:
    
-  Taxonomic Classification (Qiime 2 scikit-learn classifier)
    
-  Mapping Files for each study
    
-  List of "kit-ome" contaminants removed in each study
    
-  Raw Q2 Feature tables as .qza files

Additionally, a results folder is provided for each study with files generated during the original analysis. Note, due to the random nature of rarefaction and random forests classification, slightly different results may be generated when executing these commands

 ### Estrous Cycle Staging Study

Data Processing

In [8]:
%%bash -e

#Removing reads that don't appear in at least two samples

qiime feature-table filter-features \
  --i-table Raw_Files_Estrous_Stage/reference_hit_updated.qza \
  --p-min-samples 2 \
  --o-filtered-table Results_Files_Estrous_Stage/reference_hit_updated.qza

#Filtering out contaminants, determined from analyzing blanks

qiime feature-table filter-features \
--i-table Results_Files_Estrous_Stage/reference_hit_updated.qza \
--m-metadata-file Raw_Files_Estrous_Stage/Contaminants_All.txt \
--p-exclude-ids \
--o-filtered-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza

#Removing the mitochondria and chloroplasts

qiime taxa filter-table \
  --i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza \
  --i-taxonomy Raw_Files_Estrous_Stage/classification.qza  \
  --p-exclude mitochondria,chloroplast \
  --o-filtered-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza

#Removing blanks

qiime feature-table filter-samples \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza \
--m-metadata-file Raw_Files_Estrous_Stage/Qiime_Mapping_File.txt \
--p-where "sample_type='lavage'" \
--o-filtered-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza

#Removing a fecal-contaminated sample

qiime feature-table filter-samples \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza \
--m-metadata-file Raw_Files_Estrous_Stage/Qiime_Mapping_File.txt \
--p-where "contaminated='FALSE'" \
--o-filtered-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza


Saved FeatureTable[Frequency] to: Results_Files_Estrous_Stage/reference_hit_updated.qza
Saved FeatureTable[Frequency] to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza
Saved FeatureTable[Frequency] to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza
Saved FeatureTable[Frequency] to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza
Saved FeatureTable[Frequency] to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza


### Analysis

In [7]:
%%bash -e

#Making rarefaction plots
qiime diversity alpha-rarefaction \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza \
--p-max-depth 5000 \
--m-metadata-file Raw_Files_Estrous_Stage/Qiime_Mapping_File.txt \
--p-min-depth 100 \
--p-steps 100 \
--o-visualization Results_Files_Estrous_Stage/Alpha_Rarefaction_Plot.qzv

Saved Visualization to: Results_Files_Estrous_Stage/Alpha_Rarefaction_Plot.qzv


In [10]:
%%bash -e

#Rarifying the table

qiime feature-table rarefy \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved.qza \
--p-sampling-depth 400 \
--o-rarefied-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza

#Classifier by Estrous cycle stage

qiime sample-classifier classify-samples \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza \
--m-metadata-file Raw_Files_Estrous_Stage/Qiime_Mapping_File.txt \
--m-metadata-category stage \
--o-visualization Results_Files_Estrous_Stage/Classifier_Stage_400rare.qzv

Saved FeatureTable[Frequency] to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza
Saved Visualization to: Results_Files_Estrous_Stage/Classifier_Stage_400rare.qzv


In [14]:
#Exporting Rarified Table, 400 seqs/sample

!qiime tools export Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza --output-dir Results_Files_Estrous_Stage/


In [15]:
#Making the biom file to .txt

!biom convert -i Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.biom -o Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.txt --to-tsv

In [16]:
%%bash -e
#Alpha Diversity

qiime diversity alpha \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza \
--p-metric shannon \
--o-alpha-diversity Results_Files_Estrous_Stage/Alpha_Diversity/Rare400_Shannon.qza

qiime diversity alpha \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza \
--p-metric observed_otus \
--o-alpha-diversity Results_Files_Estrous_Stage/Alpha_Diversity/Rare400_Obs_Otus.qza



Saved SampleData[AlphaDiversity] to: Results_Files_Estrous_Stage/Alpha_Diversity/Rare400_Shann.qza
Saved SampleData[AlphaDiversity] to: Results_Files_Estrous_Stage/Alpha_Diversity/Rare400_o.qza


In [17]:
%%bash -e

#Exporting the collapsed taxa for heatmap visualizaton

qiime taxa collapse \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza \
--i-taxonomy Raw_Files_Estrous_Stage/classification.qza \
--p-level 7 \
--o-collapsed-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_L7.qza


qiime feature-table relative-frequency \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_L7.qza \
--o-relative-frequency-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_L7_Rel.qza

Saved FeatureTable[Frequency] to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_L7.qza
Saved FeatureTable[RelativeFrequency] to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_L7_Rel.qza


In [20]:
%%bash -e

qiime tools export Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_L7_Rel.qza --output-dir Results_Files_Estrous_Stage/

In [21]:
!biom convert -i Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_L7_Rel.biom -o Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_L7_Rel.txt --to-tsv

In [3]:
%%bash -e

#Beta Diversity

qiime diversity beta \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza \
--p-metric braycurtis \
--o-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza

qiime diversity beta \
--i-table Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare.qza \
--p-metric euclidean \
--o-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean.qza

Saved DistanceMatrix to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza
Saved DistanceMatrix to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean.qza


In [11]:
%%bash -e

#Significance Testing

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category stage \
--o-visualization Results_Files_Estrous_Stage/BrayCurtis_Stage_Significance.qzv

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category CST_Euc \
--o-visualization Results_Files_Estrous_Stage/BrayCurtis_CST_Significance.qzv

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category CST_Euc \
--o-visualization Results_Files_Estrous_Stage/Euclidean_CST_Significance.qzv

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category stage \
--o-visualization Results_Files_Estrous_Stage/Euclidean_Stage_Significance.qzv

Saved Visualization to: Results_Files_Estrous_Stage/BrayCurtis_Stage_Significance.qzv
Saved Visualization to: Results_Files_Estrous_Stage/BrayCurtis_CST_Significance.qzv
Saved Visualization to: Results_Files_Estrous_Stage/Euclidean_CST_Significance.qzv
Saved Visualization to: Results_Files_Estrous_Stage/Euclidean_Stage_Significance.qzv


In [1]:
%%bash -e

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category cage \
--o-visualization Results_Files_Estrous_Stage/BrayCurtis_cage_Significance.qzv

Saved Visualization to: Results_Files_Estrous_Stage/BrayCurtis_cage_Significance.qzv


In [6]:
%%bash -e

qiime diversity pcoa \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--o-pcoa Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis_pcoa.qza

qiime diversity pcoa \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean.qza \
--o-pcoa Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean_pcoa.qza

Saved PCoAResults to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis_pcoa.qza
Saved PCoAResults to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean_pcoa.qza


In [8]:
%%bash -e

qiime emperor plot \
--i-pcoa Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis_pcoa.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--o-visualization Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis_pcoa.qzv

qiime emperor plot \
--i-pcoa Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean_pcoa.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--o-visualization Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean_pcoa.qzv

Saved Visualization to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis_pcoa.qzv
Saved Visualization to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_euclidean_pcoa.qzv


In [2]:
%%bash -e

qiime tools export Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza --output-dir Results_Files_GBS_Challenge/

In [1]:
%%bash -e

qiime diversity filter-distance-matrix \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--p-where "day IN (0)" \
--o-filtered-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day0

qiime diversity filter-distance-matrix \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--p-where "day IN (3)" \
--o-filtered-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day3

qiime diversity filter-distance-matrix \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--p-where "day IN (6)" \
--o-filtered-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day6

qiime diversity filter-distance-matrix \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--p-where "day IN (9)" \
--o-filtered-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day9

qiime diversity filter-distance-matrix \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--p-where "day IN (12)" \
--o-filtered-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day12

qiime diversity filter-distance-matrix \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--p-where "day IN (15)" \
--o-filtered-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day15

Saved DistanceMatrix to: Results_Files_Estrous_Stage/Bray_Curtis_Day0.qza
Saved DistanceMatrix to: Results_Files_Estrous_Stage/Bray_Curtis_Day3.qza
Saved DistanceMatrix to: Results_Files_Estrous_Stage/Bray_Curtis_Day6.qza
Saved DistanceMatrix to: Results_Files_Estrous_Stage/Bray_Curtis_Day9.qza
Saved DistanceMatrix to: Results_Files_Estrous_Stage/Bray_Curtis_Day12.qza
Saved DistanceMatrix to: Results_Files_Estrous_Stage/Bray_Curtis_Day15.qza


In [3]:
%%bash -e

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day0.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category cage \
--o-visualization Results_Files_Estrous_Stage/Cage_d0_sig.qzv

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day3.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category cage \
--o-visualization Results_Files_Estrous_Stage/Cage_d3_sig.qzv

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day6.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category cage \
--o-visualization Results_Files_Estrous_Stage/Cage_d6_sig.qzv

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day9.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category cage \
--o-visualization Results_Files_Estrous_Stage/Cage_d9_sig.qzv

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day12.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category cage \
--o-visualization Results_Files_Estrous_Stage/Cage_d12_sig.qzv

qiime diversity beta-group-significance \
--i-distance-matrix Results_Files_Estrous_Stage/Bray_Curtis_Day15.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--m-metadata-category cage \
--o-visualization Results_Files_Estrous_Stage/Cage_d15_sig.qzv

Saved Visualization to: Results_Files_Estrous_Stage/Cage_d0_sig.qzv
Saved Visualization to: Results_Files_Estrous_Stage/Cage_d3_sig.qzv
Saved Visualization to: Results_Files_Estrous_Stage/Cage_d6_sig.qzv
Saved Visualization to: Results_Files_Estrous_Stage/Cage_d9_sig.qzv
Saved Visualization to: Results_Files_Estrous_Stage/Cage_d12_sig.qzv
Saved Visualization to: Results_Files_Estrous_Stage/Cage_d15_sig.qzv


In [9]:
%%bash -e
qiime diversity filter-distance-matrix \
--i-distance-matrix Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--p-where "group_name IN ('longitudinal')" \
--o-filtered-distance-matrix Results_Files_Estrous_Stage/Longitudinal_Bray_Curtis

Saved DistanceMatrix to: Results_Files_Estrous_Stage/Longitudinal_Bray_Curtis.qza


In [10]:
%%bash -e

qiime diversity pcoa \
--i-distance-matrix Results_Files_Estrous_Stage/Longitudinal_Bray_Curtis.qza \
--o-pcoa Results_Files_Estrous_Stage/Longitudinal_Bray_Curtis_pcoa

Saved PCoAResults to: Results_Files_Estrous_Stage/Longitudinal_Bray_Curtis_pcoa.qza


In [11]:
%%bash -e

qiime emperor plot \
--i-pcoa Results_Files_Estrous_Stage/Longitudinal_Bray_Curtis_pcoa.qza \
--m-metadata-file Results_Files_Estrous_Stage/Qiime_Mapping_File_Estrous_Staging_CST.txt \
--p-custom-axis 'day' \
--o-visualization Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis_pcoa_DAY_long.qzv

Saved Visualization to: Results_Files_Estrous_Stage/reference_hit_updated_ContaminationRemoved_400Rare_braycurtis_pcoa_DAY_long.qzv


### GBS Pathogen Challenge Study

Data Processing

In [27]:
%%bash -e

#Removing reads that don't appear in at least 2 samples

qiime feature-table filter-features \
  --i-table Raw_Files_GBS_Challenge/reference_hit_updated.qza \
  --p-min-samples 2 \
  --o-filtered-table Results_Files_GBS_Challenge/reference_hit_updated.qza

#Filtering out contaminants, determined from analyzing blanks

qiime feature-table filter-features \
--i-table Results_Files_GBS_Challenge/reference_hit_updated.qza \
--m-metadata-file Raw_Files_GBS_Challenge/Contaminants_All.txt \
--p-exclude-ids \
--o-filtered-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved.qza

#Removing the mitochondria and chloroplasts
qiime taxa filter-table \
  --i-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved.qza \
  --i-taxonomy Raw_Files_GBS_Challenge/Classification.qza \
  --p-exclude mitochondria,chloroplast \
  --o-filtered-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved.qza

#This feature table contains samples not discussed in the paper, filter to only look at the GBS and 
#PBS control mice and remove the blanks

qiime feature-table filter-samples \
  --i-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved.qza \
  --m-metadata-file Raw_Files_GBS_Challenge/Qiime_Mapping_File_GBS_Pathogen_Challenge.txt \
  --p-where "treatment_group IN ('GBS', 'PBS')" \
  --o-filtered-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS.qza


Saved FeatureTable[Frequency] to: Results_Files_GBS_Challenge/reference_hit_updated.qza
Saved FeatureTable[Frequency] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved.qza
Saved FeatureTable[Frequency] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved.qza
Saved FeatureTable[Frequency] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS.qza


In [28]:
%%bash -e

#Rarifying the table (depth determined by alpha rarefaction curves)

qiime feature-table rarefy \
--i-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS.qza \
--p-sampling-depth 500 \
--o-rarefied-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare.qza

Saved FeatureTable[Frequency] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare.qza


In [29]:
%%bash -e

qiime tools export Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare.qza --output-dir Results_Files_GBS_Challenge/

In [30]:
!biom convert -i Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare.biom -o Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare.txt --to-tsv

In [3]:
%%bash -e

#First Distances Analysis

qiime diversity beta \
--i-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare.qza \
--p-metric euclidean \
--o-distance-matrix Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_Euc.qza

qiime longitudinal first-distances \
--i-distance-matrix Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_Euc.qza \
--m-metadata-file Results_Files_GBS_Challenge/Qiime_Mapping_File_GBS_Pathogen_Challenge_CST.txt \
--p-state-column 'day' \
--p-individual-id-column 'host_subject_id' \
--o-first-distances Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_Euc_firstdist.qza

Saved DistanceMatrix to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_Euc.qza
Saved SampleData[FirstDifferences] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_Euc_firstdist.qza


In [45]:
%%bash -e

#First Distances Analysis

qiime diversity beta \
--i-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare.qza \
--p-metric braycurtis \
--o-distance-matrix Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_BrayCurtis.qza

qiime longitudinal first-distances \
--i-distance-matrix Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_BrayCurtis.qza \
--m-metadata-file Results_Files_GBS_Challenge/Qiime_Mapping_File_GBS_Pathogen_Challenge_CST.txt \
--p-state-column 'day' \
--p-individual-id-column 'host_subject_id' \
--o-first-distances Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_BrayCurtis_firstdist.qza

Saved DistanceMatrix to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_BrayCurtis.qza
Saved SampleData[FirstDifferences] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_BrayCurtis_firstdist.qza


In [ ]:
%%bash -e

qiime tools export Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_Euc_firstdist.qza --output-dir Results_Files_GBS_Challenge/

In [46]:
%%bash -e

qiime tools export Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Rare_BrayCurtis_firstdist.qza --output-dir Results_Files_GBS_Challenge/

In [27]:
%%bash -e

qiime feature-table filter-samples \
  --i-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS.qza \
  --p-min-frequency 500 \
  --o-filtered-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered.qza

Saved FeatureTable[Frequency] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered.qza


In [41]:
%%bash -e

qiime taxa collapse \
--i-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered.qza \
--i-taxonomy Raw_Files_GBS_Challenge/Classification.qza \
--p-level 7 \
--o-collapsed-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered_L7.qza

Saved FeatureTable[Frequency] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered_L7.qza


In [42]:
%%bash -e

#Add a pseudocount for log ratios

qiime composition add-pseudocount \
  --i-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered_L7.qza \
  --o-composition-table Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered_L7_pseudo.qza

Saved FeatureTable[Composition] to: Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered_L7_pseudo.qza


In [43]:
%%bash -e

qiime tools export Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered_L7_pseudo.qza --output-dir Results_Files_GBS_Challenge/

In [44]:
!biom convert -i Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered_L7_pseudo.biom -o Results_Files_GBS_Challenge/reference_hit_updated_ContaminationRemoved_GBS_PBS_500Filtered_L7_pseudo.txt --to-tsv